# 03 Modelling - GitHub Developer Engagement

### Import Libraries

In [ ]:
import os
os.environ["HADOOP_HOME"] = "S:/Courses/MIE1512/Material/Spark/winutils/"
os.environ["JAVA_HOME"] = "C:/progra~2/Java/jdk1.8.0_221/"
os.environ["JRE_HOME"] = "C:/progra~2/Java/jdk1.8.0_221/jre/"

In [ ]:
import findspark
findspark.init("S:/Courses/MIE1512/Material/Spark/spark-2.3.2-bin-hadoop2.7/")

import pyspark
sc = pyspark.SparkContext(appName="myAppName")
spark = pyspark.sql.SQLContext(sc)

In [ ]:
import pyspark.sql.types as T 
import pyspark.sql.functions as F
import seaborn as sns

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, ClusteringEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, ClusteringEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Read the Full CSV

In [ ]:
#load each table csv
all_table = spark.read.csv("full_df.csv",
                           header=True,
                           inferSchema=True)

In [ ]:
all_table.columns

['_c0',
 'ID',
 'committer_name',
 'committer_timestamp',
 'repo_names',
 'repo_org',
 'no_change_periods',
 'tot_repo_comm_absent',
 'total_repo_comm_new',
 'agg_absent_periods',
 'new_ppl_periods',
 'avg_multi_ratio',
 'y']

In [ ]:
#there is a _c0 column which looks questionable. Delete this col and inspect preview of all_table_14 to verify join.
all_table_1 = all_table.select([c for c in all_table.columns if c not in 
                                    {"ID", "committer_name", "committer_timestamp", 
                                       "repo_names","repo_org", "first_commit", "_c0"
                                    }])

In [ ]:
all_table_1.columns

['no_change_periods',
 'tot_repo_comm_absent',
 'total_repo_comm_new',
 'agg_absent_periods',
 'new_ppl_periods',
 'avg_multi_ratio',
 'y']

In [ ]:
#looks good
all_table_1.limit(5).toPandas()

,no_change_periods,tot_repo_comm_absent,total_repo_comm_new,agg_absent_periods,new_ppl_periods,avg_multi_ratio,y
0,0,19,7,5,3,0.166667,disengaged
1,0,61,29,5,5,0.500000,disengaged
2,0,61,29,5,5,0.500000,disengaged
3,0,61,29,5,5,0.500000,disengaged
4,0,61,29,5,5,0.500000,disengaged


# MODELLING

In [ ]:
numericCols = ['no_change_periods',
 'tot_repo_comm_absent',
 'total_repo_comm_new',
 'agg_absent_periods',
 'new_ppl_periods',
 'avg_multi_ratio']

In [ ]:
stages = []

label_stringIdx = StringIndexer(inputCol = 'y', outputCol = 'label')
stages += [label_stringIdx]

assemblerInputs = numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
stages

[StringIndexer_4eeab5fde6995d5e6717, VectorAssembler_4c85aa3d6af6ca5a5958]

In [ ]:
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(all_table_1)


In [ ]:
all_data_transformed = pipelineModel.transform(all_table_1)

In [ ]:
selectedCols = ['label', 'features']
all_data_transformed = all_data_transformed.select(selectedCols)

In [ ]:
train, test = all_data_transformed.randomSplit([0.7, 0.3], seed = 1000)

In [ ]:
# train.count(), test.count()

(733710, 314826)

# Logistic Regression Model

In [ ]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [ ]:
predictions = lrModel.transform(test)
evaluator = BinaryClassificationEvaluator().setMetricName("areaUnderROC")
evaluator.evaluate(predictions)

0.5283711630876493